# Clean data to prepare for NLP

##### Import libraries

In [16]:
import pandas as pd
import numpy as np

##### Load data

In [18]:
# PreCovid Reddit posts
PreCovid_posts = pd.read_csv('../data/raw/reddit_posts_PreCovid.csv') 

# PostCovid Reddit posts
PostCovid_posts = pd.read_csv('../data/raw/reddit_posts_PostCovid.csv') 

# Clean data

In [19]:
PreCovid_posts.head()

,author,title,selftext,created_utc,subreddit
0,sub16784,Help. Dying inside,So I posted something similar on the binge eat...,1526608139,bulimia
1,sub12545,Book Recommendations? - Brain Over Binge,"Hi, a few months ago I went to an Eating Disor...",1526595326,bulimia
2,sub10227,In need of some encouragement,"I just binged on pizza rolls, doritos and cand...",1526528018,bulimia
3,sub10942,Struggling so badly. I can't take this feeling,I had a surgery that doesn't allow me to purg...,1526519003,bulimia
4,sub4221,Been trying to quit this nasty habit...,Struggling with bulimia for 4 years now and it...,1526360915,bulimia


In [20]:
# Double check - Did I get the comments but not the posts themsleves?
PostCovid_posts.head()

,author,title,selftext,created_utc,subreddit
0,sub17706,if i was able to keep an excess of food in my ...,NaN,1593568164,bulimia
1,sub19142,Get help. Please.,I know it’s embarrassing. I know it is hard to...,1593566253,bulimia
2,sub15645,More bulimemes for you guys 👌🏻,NaN,1593556888,bulimia
3,sub10794,Teeth Are Rough?,[Here are my teeth](https://imgur.com/a/pyV5Ss...,1593555043,bulimia
4,sub17937,Badanie,[removed],1593534095,bulimia


In [21]:
PreCovid_posts.dtypes

author         object
title          object
selftext       object
created_utc     int64
subreddit      object
dtype: object

In [22]:
PostCovid_posts.dtypes

author         object
title          object
selftext       object
created_utc     int64
subreddit      object
dtype: object

In [25]:
len(PreCovid_posts) - len(PostCovid_posts)

2079

#### Check Null Values

In [26]:
PreCovid_posts.isnull().sum()

author            0
title             0
selftext       4002
created_utc       0
subreddit         0
dtype: int64

In [27]:
PostCovid_posts.isnull().sum()

author            0
title             0
selftext       2910
created_utc       0
subreddit         0
dtype: int64